<a href="https://colab.research.google.com/github/OmniDevsOficial/API-Censo-2022/blob/feature%2FOesteSul/PlanejaSJC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bem-Vindo ao Planeja SJC!

Aqui, você encontrará uma ampla gama de recursos para explorar e compreender a dinâmica da nossa cidade. Nossa plataforma oferece mapas interativos que permitem visualizar diferentes aspectos do espaço urbano, desde a infraestrutura até as áreas verdes, ajudando você a entender melhor o ambiente em que vive. Além disso, disponibilizamos dados demográficos detalhados, que fornecem uma visão abrangente sobre a população, suas características e necessidades. Esses dados são essenciais para que cidadãos, planejadores urbanos e gestores públicos possam tomar decisões informadas e embasadas.

### Sumário dos Gráficos

1 - População Total por Bairro e Zona

2 - Crescimento Populacional por Zona entre 2010 e 2022

3 - Pirâmide Etária

4 - Densidade Populacional por Região

### Dependências do Projeto

In [12]:
import os
from contextlib import redirect_stdout
import io

# Define o diretório atual como o /content
os.chdir('/content')

# Baixa os arquivos do repositório Github para o /content do Colab
with redirect_stdout(io.StringIO()):
    !git clone https://github.com/OmniDevsOficial/API-Censo-2022.git

# Define a pasta onde estão os arquivos da base de dados
data_path = '/content/API-Censo-2022/Backend/database/'

# Realiza as importações para ler arquivos e mostrar gráficos
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

### População Total por Bairro e Zona

In [13]:
#puxa a tabela de Pop por bairro e região do github e atribui as variaveis "Pop_Tabela_Bairro" e "Pop_Tabela_Regiao".
Pop_Tabela_Bairro= pd.read_excel(data_path + 'populacao_subsetores_completo.xlsx')

Pop_Tabela_Regiao= pd.read_excel(data_path + "censo_2022.xlsx", sheet_name="População por Região")

# Retira o Total Geral, que é referente a cidade e não uma região
Pop_Tabela_Regiao = Pop_Tabela_Regiao[Pop_Tabela_Regiao['Região'] != 'Total Geral']

## removido da vizualição devido a repetição de informação.
##cria um gráfico em relação as tabelas, nesse caso a tabela censo_2022.
#Pop_Total_Regiao=px.bar(Pop_Tabela_Regiao, x="Região", y="População", color="Região")
#Pop_Total_Regiao.show()

Pop_Bairro_Centro=px.bar(Pop_Tabela_Bairro, x="Subsetor – Nome", y="População", color="Região")

# Defina a lista de regiões que você quer exibir inicialmente
regioes_inicialmente_visiveis = ['Centro', 'Sul'] # Exemplo: inicie mostrando Centro e Sul

# Esconda as traces que não estão nas regiões inicialmente visíveis
for trace in Pop_Bairro_Centro.data:
    region_setor = trace.name # O nome da trace é o valor da coluna 'Região_Setor'
    region = region_setor.split(' - ')[0] # Extrai a região

    if region not in regioes_inicialmente_visiveis:
        trace.visible = 'legendonly' # Esconde a trace, mas mantém na legenda

Pop_Bairro_Centro.update_layout(
    legend_title_text='Região',
    title='População por Bairro e Região',
    yaxis_title='População',
    xaxis_title='Bairros',
    plot_bgcolor='lightgray'
)

Pop_Bairro_Centro.show()

In [14]:
regioes_unicas = Pop_Tabela_Bairro['Região'].unique().tolist()

for regiao in regioes_unicas:
  df_regiao = Pop_Tabela_Bairro[Pop_Tabela_Bairro['Região'] == regiao].copy()
  regioes = px.bar(
      df_regiao,
      x='Subsetor – Nome',
      y='População',
      title=f'População por Bairro na Região: {regiao}'
  )
  regioes.update_layout(xaxis_title='Bairros')
  if regiao == "São Francisco Xavier":
    regioes.update_traces(width=0.10)
  regioes.show()

### Crescimento Populacional por Zona entre 2010 e 2022

In [15]:
# o colab faz a leitura da tabela e mostra um display dela
file_path = data_path + 'tabel_pop.xlsx'

df = pd.read_excel(file_path)
# Ordenar o DataFrame pela população de 2022
df_sorted = df.sort_values(by='Pop. 2022', ascending=False)

fig = px.bar(
    df_sorted,
    x='Região',
    y=['Pop. 2010', 'Pop. 2022'],
    barmode='group', # Agrupa as Colunas
    title='Crescimento Populacional por Região (2010 vs 2022)',
)

fig.update_layout(
    template='plotly_white',
    title_x=0.5,
    yaxis_title='Número de Habitantes',
    xaxis_title=None,
    legend_title='Ano do Censo'
)

fig.show()

df['Crescimento_Percentual'] = df['Crescimento %'] * 100

#Aqui eu crio a varável de crescimento, leio o arquivo ja colocado anteriormente no códico e começo a estruturar o gráfico
crescimento=px.bar(df, x= "Região", y= "Crescimento_Percentual",
color="Região",
title= "Taxa de Crescimento Populacional",
text= "Crescimento_Percentual"
)

# Adicione o símbolo de '%' e defina a formatação
crescimento.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

#Essa parte serve pra estilizar o gráfico e seu layout
crescimento.update_layout(
    template='plotly_white',
    title_x=0.5,
    yaxis_title='Percentual de Habitantes',
    xaxis_title=None,
    legend_title='Regiões'
)

crescimento.show()

In [16]:
regioes_unicas = df['Região'].unique().tolist()

for regiao in regioes_unicas:
  df_regiao = df[df['Região'] == regiao].copy()

  # Calculate Crescimento_Percentual for the regional DataFrame
  df_regiao['Crescimento_Percentual'] = df_regiao['Crescimento %'] * 100

  # Cria um gráfico por população (2010 vs 2022).
  fig_pop = px.bar(
      df_regiao,
      x='Região',
      y=['Pop. 2010', 'Pop. 2022'],
      barmode='group',
      title=f'População por Região: {regiao} (2010 vs 2022)',
      labels={'value': 'Número de Habitantes', 'variable': 'Ano do Censo'}
  )
  fig_pop.update_layout(xaxis_title=None, legend_title='Ano do Censo', bargap=0.8)
  fig_pop.update_traces(width=0.10)
  fig_pop.show()

  # Cria um gráfico por porcetagem de crescimento.
  fig_growth = px.bar(
      df_regiao,
      x='Região',
      y='Crescimento_Percentual',
      title=f'Taxa de Crescimento Populacional: {regiao} (2010-2022)',
      labels={'Crescimento %': 'Percentual de Crescimento'}
  )

  fig_growth.update_layout(xaxis_title=None, yaxis_title='Percentual de Crescimento (%)')
  fig_growth.update_traces(width=0.10)
  fig_growth.show()

### Pirâmide Etária

In [42]:
# Importa a tabela da faixa etária 2022
etaria_path = data_path + 'faixa_etaria.xlsx'
pk = pd.read_excel(etaria_path)

# Exclui a última coluna - Total 2022
pk_ultima_coluna = pk.drop(columns = ['Total'])
# Define o nome das colunas 2022
pk_ultima_coluna.columns = 'Idade', 'Homens', 'Mulheres'

idades = ['100 anos ou mais', '95 a 99 anos', '90 a 94 anos',
          '85 a 89 anos', '80 a 84 anos', '75 a 79 anos', '70 a 74 anos',
          '65 a 69 anos', '60 a 64 anos', '55 a 59 anos', '50 a 54 anos',
          '45 a 49 anos', '40 a 44 anos', '35 a 39 anos', '30 a 34 anos',
          '25 a 29 anos', '20 a 24 anos', '15 a 19 anos', '10 a 14 anos',
          '5 a 9 anos', '0 a 4 anos']
idades_invertidas = idades[::-1]

faixa_etaria_homens = np.array(pk_ultima_coluna['Homens'])
faixa_etaria_mulheres = np.array(pk_ultima_coluna['Mulheres'])

piramide = go.Figure()

# Gráfico de colunas horizontais - Mulheres
piramide.add_trace(go.Bar(x = faixa_etaria_mulheres,
                          y = idades_invertidas,
                          name = 'Mulheres 2022',
                          orientation = 'h',
                          marker = dict(color = '#FF69B4'),
                          hovertemplate = 'Idade: %{y} <br><b>%{customdata} Mulheres</b><extra></extra>',
                          customdata = np.abs(faixa_etaria_mulheres)))

# Gráfico de colunas horizontais - Homens
piramide.add_trace(go.Bar(x = -faixa_etaria_homens,
                          y = idades_invertidas,
                          orientation = 'h',
                          name='Homens 2022',
                          marker = dict(color = '#6495ED'),
                          hovertemplate = 'Idade: %{y} <br><b>%{customdata} Homens</b><extra></extra>',
                          customdata = np.abs(faixa_etaria_homens)))
piramide.update_layout(
    title='Pirâmide Etária de São José dos Campos 2022',
    title_x = 0.5,
    barmode = 'overlay',
    xaxis = dict(tickvals = [-30000, -25000, -20000, -15000, -10000, -5000, 0, 5000, 10000, 15000, 20000, 25000, 30000],
                 ticktext = [30000, 25000, 20000, 15000, 10000, 5000, 0, 5000, 10000, 15000, 20000, 25000, 30000]),
    plot_bgcolor = 'white')
piramide.show()


# Caso precise mudar a forma que mostra os gráficos. Aqui abaixo fica o dados de 2010


# Importa a tabela da faixa etária 2010
etaria_2010_path = data_path + 'faixa_etaria_2010.xlsx'
piramide_2010 = pd.read_excel(etaria_2010_path)

# Exclui colunas do censo de 2010
coluna_piramide_2010 = piramide_2010.drop(columns = ['Distribuição', 'Sexo'])
# Define o nome das colunas do censo de 2010
coluna_piramide_2010.columns = 'Idade', 'Homens', 'Mulheres'

idades = ['100 anos ou mais', '95 a 99 anos', '90 a 94 anos',
          '85 a 89 anos', '80 a 84 anos', '75 a 79 anos', '70 a 74 anos',
          '65 a 69 anos', '60 a 64 anos', '55 a 59 anos', '50 a 54 anos',
          '45 a 49 anos', '40 a 44 anos', '35 a 39 anos', '30 a 34 anos',
          '25 a 29 anos', '20 a 24 anos', '15 a 19 anos', '10 a 14 anos',
          '5 a 9 anos', '0 a 4 anos']

# Colunas Homens e mulheres 2010
faixa_etaria_homens_2010 = np.array(coluna_piramide_2010['Homens'])
faixa_etaria_mulheres_2010 = np.array(coluna_piramide_2010['Mulheres'])

# Importa a tabela da faixa etária 2010
etaria_2010_path = data_path + 'faixa_etaria_2010.xlsx'
piramide_2010 = pd.read_excel(etaria_2010_path)

# Exclui colunas do censo de 2010
coluna_piramide_2010 = piramide_2010.drop(columns = ['Distribuição', 'Sexo'])
# Define o nome das colunas do censo de 2010
coluna_piramide_2010.columns = 'Idade', 'Homens', 'Mulheres'

piramide = go.Figure()

# Gráfico de colunas horizontais - Mulheres 2010
piramide.add_trace(go.Bar(x = faixa_etaria_mulheres_2010,
                          y = idades,
                          name = 'Mulheres 2010',
                          orientation = 'h',
                          marker = dict(color = 'violet'),
                          hovertemplate = 'Idade: %{y} <br><b>%{customdata} Mulheres</b><extra></extra>',
                          customdata = np.abs(faixa_etaria_mulheres_2010)))

# Gráfico de colunas horizontais - Homens 2010
piramide.add_trace(go.Bar(x = -faixa_etaria_homens_2010,
                          y = idades,
                          orientation = 'h',
                          name='Homens 2010',
                          marker = dict(color = '#1E90FF'),
                          hovertemplate = 'Idade: %{y} <br><b>%{customdata} Homens</b><extra></extra>',
                          customdata = np.abs(faixa_etaria_homens_2010)))
piramide.update_layout(
    title='Pirâmide Etária de São José dos Campos 2010',
    title_x = 0.5,
    barmode = 'overlay',
    xaxis = dict(tickvals = [-30000, -25000, -20000, -15000, -10000, -5000, 0, 5000, 10000, 15000, 20000, 25000, 30000],
                 ticktext = [30000, 25000, 20000, 15000, 10000, 5000, 0, 5000, 10000, 15000, 20000, 25000, 30000]),
    plot_bgcolor = 'white')
# Impede que a pirâmide fique de cabeça para baixo
piramide.update_yaxes(autorange='reversed')
piramide.show()

### Densidade Populacional por Região

In [18]:
excel_file = data_path + 'tabel_pop.xlsx'

df_pop = pd.read_excel(excel_file, sheet_name=0,  index_col=0,)

# Áreas(km²) das Regiões de SJC obtidas em: https://planodiretor.sjc.sp.gov.br/resources/uploads/EstudoTecnico/Anexo/PD_VOLUME_1_C7_REGIOES_URBANAS.pdf
areas = {
    "Norte": 63.73,
    "Sul": 56.51,
    "Leste": 134.69,
    "Oeste": 44.01,
    "Sudeste": 35.66,
    "Centro": 18.68,
    # "Rural": 745.7,
}
# Transforma o dicionário das áreas em um Dataframe
df_area = pd.DataFrame(list(areas.items()), columns=['Região', 'Área (km²)'])

# Une os dados de população e área
df = pd.merge(df_pop, df_area, on='Região')

# Retira os dados da Região Rural, pois possui densidade muito pequena para ser vista no gráfico
df = df[df['Região'] != 'Rural']

# Calcula a densidade populacional
df['2010'] = df['Pop. 2010'] / df['Área (km²)']
df['2022'] = df['Pop. 2022'] / df['Área (km²)']

# Ordena da maior densidade de 2022 para a menor
df_sorted = df.sort_values(by='2022', ascending=False)

# Organiza como mostrar e quais informações serão mostradas
fig = px.bar(
    df_sorted,
    x='Região',
    y=['2010', '2022'],
    barmode='group',
    title='Densidade Populacional por Região (2010 x 2022)',
    labels={'value': 'Densidade Populacional (hab/km²)', 'Região': 'Região', 'variable': 'Ano'},
)
fig.update_layout(legend_title_text='Ano', xaxis_title=None)

fig.show()

In [19]:
regioes_unicas = df['Região'].unique().tolist()

for regiao in regioes_unicas:
  df_regiao = df[df['Região'] == regiao].copy()

  fig = px.bar(
      df_regiao,
      x='Região',
      y=['Pop. 2010', 'Pop. 2022'],
      barmode='group',
      title=f'Densidade Populacional para a Região: {regiao}',
      labels={'Densidade Populacional': 'Densidade Populacional (hab/km²)'}
  )
  fig.update_layout(yaxis_title='Densidade Populacional (hab/km²)', bargap=0.8)
  fig.update_traces(width=0.10)
  fig.show()

###Nivel de Escolaridade

In [20]:
escolaridade22= pd.read_excel(data_path + 'Censo 2022 - Escolaridade - SJC e SP.xlsx')

#display(escolaridade22)

Nivel_Escolaridade=px.bar(
    escolaridade22,
    x='Escolaridade',
    y='São José dos Campos ',
    title='Nivel de Escolaridade em SJC 2022'
)
Nivel_Escolaridade.update_layout(yaxis_title='População')

Pop_total=697054

escolaridade22['São José dos Campos (%)'] = (escolaridade22['São José dos Campos '] / Pop_total) * 100

Nivel_Escolaridade_Porc=px.bar(
    escolaridade22,
    x='Escolaridade',
    y='São José dos Campos (%)',
    title='Nivel de Escolaridade em SJC 2022 (%)'
)

# Adicione o símbolo de '%' e defina a formatação
Nivel_Escolaridade_Porc.update_traces(texttemplate='%{y:.2f}%', textposition='outside')
Nivel_Escolaridade_Porc.update_layout(yaxis_title='População (%)', yaxis=dict(range=[0, 35]))

Nivel_Escolaridade.show()
Nivel_Escolaridade_Porc.show()

### Internet por domicílio

In [21]:
internet= pd.read_excel(data_path + 'Censo 2022 - Uso de internet dentro do domicílio - SJC e Brasil.xlsx', sheet_name="Sheet2")
#internet = internet.drop(columns=['Unnamed: 6', 'Fonte: censo2022.ibge.gov.br/panorama', 'Sigla UF', 'Código do Município'])

#display(internet)

internet_porpocao=px.bar(
    internet,
    x='Município',
    y=['Sim', 'Não'],
    barmode='group',
    title='Internet por domicílio 2022',
    labels={'value': 'População (%)', 'Município': 'Município/Pais', 'variable': 'Uso de Internet'}
)
internet_porpocao.update_traces(texttemplate='%{y:.2f}%', textposition='outside')
internet_porpocao.update_layout(yaxis_title='Percentual (%)', xaxis_title='Município/Pais')
internet_porpocao.show()


Crescimento Populacional Oeste e Sul

In [50]:
zonas = pd.read_excel(data_path + 'oeste_sul.xlsx')

regioes = px.bar(
    zonas,
    y=['Pop. 2010', 'Pop. 2022'],
    x=['Oeste', 'Sul'],
    barmode='group',
    title='Taxa de Crescimento – Sul vs Oeste',
    labels={'value': 'População', 'variable': 'Ano'}
)

# Estilização leve
regioes.update_traces(marker_line_width=1.2, marker_line_color='black')

regioes.update_layout(
    title_x=0.5,
    font=dict(size=14),
    xaxis_title=None,
    yaxis_title='População',
    plot_bgcolor='white',
    legend_title_text='Ano',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    )
)

regioes.show()
